# Izpit: Analitika 2: Strojno učenje v Python-u

Rok oddaje: `TODO!`

Cilj: `doseči čim boljšo končno napoved.`

Za vsa vprašanja smo na voljo.

Lahko si pomagate z uporabo gradiv in internetom. Ne pozabite na uradno dokumentacijo.

Srečno!!

## Izbirate lahko med dvema nalogama oziroma problemoma:

* Klasifikacijski --> Glede na podane karakteristike površja, klasificirajte rastje se tam nahaja
* Regresijski --> Napoved cene hiše

# Problem 1: Klasifikacijski

Podane imate podatke o _30m x 30m_ območjih divjine/narave in njihovih karakektaristikah.

Atributi:
* `NadmorskaVisina` (nadmorska višina v metrih)
* `StopinjeAzimuth` (azimut kot v stopinjah)
* `Naklon` (naklon območja v stopinjah)
* `DolzinaDoVode` (najkrajša dolžina do vode na površju v metrih)
* `VertikalnaDolzinaDoVode` (vertikalna najkrajša dolžina do vode na površju v metrih)
* `DolzinaDoZeleznice` (najkrajša dožina do železnice v metrih)
* `HillshadeIndeksOb9h` (hillshade indeks ob 9:00 --> območe vrednosti: [0,255])
* `HillshadeIndeksOb12h` (hillshade indeks ob 12:00 --> območe vrednosti: [0,255])
* `HillshadeIndeksOb15h` (hillshade indeks ob 15:00 --> območe vrednosti: [0,255])
* `DolzinaDoPozarneTocke` (najkrajša dolžina do požarno nevarne/vnetljive točke v metrih)
* `Obmocje` (indeks območja v katerem se nahaja ta predel --> območje vrednosti: [1,4])
* `TipZemlje` (indeks tipa zemlje na tem območju --> območje vrednosti: [1,40])


Ciljni atribut:
* `TipRastja` (indeks tipa rastja ki se nahaja na tem območju)

Vaša naloga je, da izdelate klasifikacijski model, ki bo čim boljše klasificiral tip rastja, ki se nahaja na nekem območju glede na podane atribute.

Za napovedovanje lahko uporabite kakršnekoli metode.

Pred samo napovedjo boste morali značilke urediti v obliko, ki bo omogočala napovedovanje tipa rastja.

```
POZOR: Pazite, da za optimizacijo modela uporabljate validacijske podatke, ki jih naredite z delitvijo train podatkov, že naloženih s spodnjo kodo. Spodnje celice ne spreminjajte, da vsi primerjamo rezultate z istimi testnimi podatki.
```

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv("./data/Problem1_tip_narave.csv",sep="\t",encoding="utf-8")
train, test = train_test_split(data, test_size=0.3, random_state=42, shuffle=True)
# display(data)
# display(train)
# display(test)

In [ ]:
from sklearn.preprocessing import StandardScaler

# Train in test podatke razdelimo na znacilke in prediktorje za nadaljnjo uporabo
y_train = train['TipRastja']
X_train = train.drop(columns=["TipRastja"])
y_test = test['TipRastja']
X_test = test.drop(columns=["TipRastja"])

# Pripravimo si skalirane podatke, ki jih uporabljamo v nadaljevanju notebooka
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

#### Data study before building model

In [ ]:
# OPAZKE
# - Ni manjkajočih podatkov, kar pomeni da imputacija ne bo potrebna
# - Prav tako opazimo, da so vse značilke tipa float64 (predictor je int). To pomeni, da encoding ni potreben. 
display(data.info())

In [ ]:
display(data.describe().transpose())

**Porazdelitev vrednosti v značilkah**
- Ugotovitev: v primeru uporabe nekaterih modelov (npr: logistična regresija) bi bilo smiselno normalizirati porazdelitve nekaterih značilk.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

data.hist(bins=30, figsize=(12,12), density=True)
plt.show()

Podobno ugotovimo z uporabo qq-plota

In [ ]:
fig, axes = plt.subplots(nrows=len(data.columns), ncols=1, figsize=(6, 30))

for i, col in enumerate(data.columns):
    ax = axes[i]
    stats.probplot(data[col], dist="norm", plot=ax)
    ax.set_title(f"Probability Plot for {col}")

plt.tight_layout()
plt.show()

- magnitude spremenljivk

In [ ]:
data.max() - data.min()
# magnitude dolocenih spremenljivk se zelo razlikujejo - potrebno skaliranje

#### Model building

##### Logistična regresija

- zaradi večjega števila sample-ov uporabimo metodo 'newton-cholesky'
    - `newton-cholesky` is a good choice for n_samples >> n_features

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression

traning_scores = {}
testing_scores = {}

c_values = np.logspace(-3, 9, num=13)

for c in c_values:
    logreg_diff = LogisticRegression(C=c, solver='newton-cholesky',max_iter=100).fit(X_train_scaled, y_train)
    traning_scores[c] = logreg_diff.score(X_train_scaled, y_train)
    testing_scores[c] = logreg_diff.score(X_test_scaled, y_test)
    
plt.plot(traning_scores.keys(), traning_scores.values(), c="blue", label="training")
plt.plot(testing_scores.keys(), testing_scores.values(), c="red", label="testing")
plt.legend()
plt.xscale('log')
plt.show()

# display(traning_scores)
# display(testing_scores)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
logreg = LogisticRegression(C=10, solver='newton-cholesky',max_iter=10000)
logreg.fit(X_train_scaled, y_train)

predictions = logreg.predict(X_test_scaled)
print(f"Training set score: {logreg.score(X_train_scaled, y_train):.2f}")
print(f"Test set score: {logreg.score(X_test_scaled, y_test):.2f}")
print(f"Accuracy: {accuracy_score(y_test, predictions):.2f}")
print(confusion_matrix(y_test, predictions))

Sklep: Model logistične regresije ni dovolj kompleksen, underfitta za ta primer. Za boljše rezultate bi bilo potrebno linearizirati določene značilke. V nadaljevanju raje poskusimo z drugimi klasifikacijskimi metodami.

##### KNeighbour and Decision tree

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

def get_score(*, model, X_train, X_test, y_train, y_test):
    predictions = model.predict(X_test)
    print(f"------------- {type(model).__name__} -------------")
    print(f"Training set score: {model.score(X_train, y_train):.2f}")
    print(f"Test set score: {model.score(X_test, y_test):.2f}")
    print('Accuracy: ', accuracy_score(y_test, predictions))
    print(confusion_matrix(y_test, predictions))

knn = KNeighborsClassifier(n_neighbors=5, n_jobs=-1)
knn.fit(X_train_scaled, y_train)
get_score(model=knn, X_train=X_train_scaled, X_test=X_test_scaled, y_train=y_train, y_test=y_test)


In [ ]:
tree = DecisionTreeClassifier()
tree.fit(X_train, y_train)
get_score(model=tree, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test)

##### Random Forest
- uporabimo hiperoptimizacijo modela v kombinaciji s cross-validacijo

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import optuna

# forest = RandomForestClassifier(n_estimators=100, n_jobs=-1)
# forest.fit(X_train, y_train)
# get_score(model=forest, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test)

def objective(trial, X_train, y_train):
    n_estimators = trial.suggest_int('n_estimators', 2, 20)
    max_depth = int(trial.suggest_float('max_depth', 1, 32, log=True))    
    clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)    
    return cross_val_score(clf, X_train, y_train, n_jobs=-1, cv=3).mean()

func = lambda trial: objective(trial, X_train, y_train)

study = optuna.create_study(direction='maximize') # povemo ali zelimo minimizirati ali maximizirati
study.optimize(func, n_trials=100)
trial = study.best_trial

Best results
```
Accuracy: 0.9461431781759922
Best hyperparameters: {'n_estimators': 19, 'max_depth': 31.05729835942615}
```

In [ ]:
trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_slice(study)

In [ ]:
optuna.visualization.plot_contour(study, params=['n_estimators', 'max_depth'])

##### XGBoost

Ne pozabi?
 - ali je potreben feature engineering
 - grid search in cross validation
 - na koncu uporabi metode za oceno uspešnosti modela
 - uporabi pipeline
 - zmanjšanje števila značilk?
 - sklairanje
 - normalizacija 
 - potrebno je regularizirati model zaradi velikega števila spremenljivk da ne pride do overfittinga